# Scope AMR-to-LF

This notebook runs through the scope version of the translation function in section 4 of the paper. World variables are not represented as they are retrivable from the semantic type of AMR constants translated into STLC.

`believe-01`, `sick-05`, etc. : $e \rightarrow s \rightarrow t$

`ARG0`, `ARG1`, etc. : $e \rightarrow e \rightarrow s \rightarrow t$

`cont` : $e \rightarrow (s \rightarrow t) \rightarrow s \rightarrow t$

`every`, `some`, `a`, etc. : $(e \rightarrow s \rightarrow t) \rightarrow (e \rightarrow s \rightarrow t) \rightarrow s \rightarrow t $

`&` : $(s \rightarrow t) \rightarrow (s \rightarrow t) \rightarrow s \rightarrow t$

In [1]:
import penman

## Instance Assignment
Instance assignment adds the variable name to free. Uninstantiated variables are not added to free.

In [2]:
def instance_assignment(triple, free):
    lf = f'{triple[2]}({triple[0]})'
    free.add(triple[0])
    return lf, free

In [3]:
amr = '(x / P)'
g = penman.decode(amr).triples

instance_assignment(g[0], set())

('P(x)', {'x'})

## Role Assignment
This rule is simpler than in the paper for two reasons:

- We do not need the lambda term in the simple case.
- We do not need a second rule for when the second argument is complex.

This is because we are working with `amr.triples` which specifies the first argument of the relation.

In [4]:
def role_assignment(triple):
    lf = f'{triple[1][1:]}({triple[0]})({triple[2]})'
    return lf

In [5]:
free = set()
amr = '(x / P :R y)'
g = penman.decode(amr).triples

role_assignment(g[1])

'R(x)(y)'

## Close

`close()` takes a tuple of an lf and the set of instantiated free variables, and a world index.
- It existentially closes the free variables and removes them from the set.
- It introduces lambda abstraction over the world variables within the lf.

In [6]:
def close(lf, free, store):
    free -= set([x for x in store])
    if free:
        prefix = f'∃' + ','.join(f'{x}' for x in free) + '.'
        free = set()
        lf = prefix + lf
    return lf, free, store

In [7]:
close('P(x) & Q(y)', set(['x', 'y']), {'x' : 'every(boy)'})

('∃y.P(x) & Q(y)', set(), {'x': 'every(boy)'})

## Tree Depth

In addition to computing the scope of **:content** we reference the depth of the parent node for complex restrictor arguments of generalized quantifiers (GQs) as well as **:pred** arguments of scope node.

In [8]:
def node_depth(epidata):
    '''
    Uses penman epidata to determine node depth
    Returns dictionary {triple : depth} for triple in epidata
    where depth = depth of first node in triple
    '''
    depth = 0
    tree_depth = {}
    
    for triple in epidata:
        tree_depth[triple] = depth
        if epidata[triple]:
            for stack_op in epidata[triple]:
                if isinstance(stack_op, penman.layout.Push):
                    depth += 1
                elif isinstance(stack_op, penman.layout.Pop):
                    depth -= 1
    return tree_depth

## Quantifier storage & Scope assignment

`quant_store()` and `scope_assignment()` are defined along with `cont_assignment` in the same cell as `translation_func()` as they each call `translation_func()` to translate subgraphs.

- `quant_store()` translates AMR quantifiers into GQs, removes them from the graph, and adds them to a store.

- `translation_func()` iterates through the list of triples and calls either `instance_assignment()`, `role_assignment()`, `cont_assignment()` or `scope_assignment()`.

- `scope_assignment()` interprets scope nodes.

- `cont_assignment()` identifies the subgraph which forms the scope of **:content** and applies `translation_func()` to that subgraph.


In [9]:
def quant_store(graph, tree_depth):
    '''
    Takes a graph and tree_depth information.
    Parses the graph and stores AMR quantifiers as GQs.
    Removes all GQs from the graph.
    Returns the graph and a store.
    '''

    store = {}

    for triple in graph:
        if triple[1] == ':quant':
            var = triple[0]

            # Find triple and index for start of restrictor argument
            for triple2 in graph:
                if (triple2[0] == var) & (triple2[1] == ':instance'):
                    restrictor = triple2
                    i_restrictor = graph.index(restrictor)
                    break

            # Build restrictor subgraph by iterating through remaining triples, if depth < triple, break
            subgraph = []
            for remaining_triple in graph[i_restrictor:]:
                if tree_depth[remaining_triple] < tree_depth[restrictor]:
                    break
                else:
                    subgraph.append(remaining_triple)

            # Translate subgraph
            restrictor_lf, restrictor_free, store = translation_func([x for x in subgraph if x[1] != ':quant'], set(), tree_depth, store)

            # Don't add var to free in restrictor
            restrictor_free -= set(var)

            # Store GQ in store
            lf = close(restrictor_lf, restrictor_free, store)[0]
            store = close(restrictor_lf, restrictor_free, store)[2]
            store[var] = f'{triple[2]}(λ{var}.{lf})'

            # Remove GQs from the graph
            for sub_triple in subgraph:
                graph.remove(sub_triple)

    return graph, store


def scope_assignment(graph, i, tree_depth, store):
    ''' 
    Takes graph, index of scope node, tree_depth info, and store.
    Determines scope order of GQs. Translates scope node's :pred argument.
    Pops GQs from the store in order.
    Returns LF, store, and index for end of subgraph.
    '''

    # Order of GQ scope (in case they do not appear in order)
    scope_order = []
    for triple in [x for x in graph if (x[0] == graph[i][0]) & (':ARG' in x[1])]:
        scope_order.append(triple)
    scope_order.sort(key = lambda x: x[1], reverse = True)

    # Find start of scope node :pred argument
    for triple in graph[i:]:
        if triple[0:2] == (graph[i][0], ':pred'):
            pred_index = graph.index(triple)
            
    # Build :pred subgraph by iterating through remaining triples, if depth < or = that of :pred triple, break
    subgraph = []
    i = pred_index + 1

    for remaining_triple in graph[i:]:
        if tree_depth[remaining_triple] <= tree_depth[graph[pred_index]]:
            break
        else: 
            subgraph.append(remaining_triple)
            i += 1
    
    lf, scope_free, store = translation_func(subgraph, set(), tree_depth, store)
    lf_scope = close(lf, scope_free, store)[0]

    # Pop GQs from store in order
    if scope_order:
        for triple in scope_order:
            var = triple[2]
            gq = store[var]
            lf_scope = f'{gq}(λ{var}.{lf_scope})'
            store.pop(var)

    return lf_scope, store, i


def translation_func(graph, free, tree_depth, store):
    ''' 
    Takes graph, set of free instantiated variables, tree_depth info, and store.
    Cycles through triples in graph, and applies either scope assignment, 
    instance assignment,content assignment, or role assignment.
    Builds LF via conjunction.
    Returns LF, free set, and store.
    '''
    
    lf = ''
    i = 0

    while i < len(graph):
        triple = graph[i]

        # scope assignment =
        if triple[2] == 'SCOPE':
            scope_lf, store, i = scope_assignment(graph, i, tree_depth, store)
            lf += ' & ' + scope_lf
            i += 1

        # instance assignment
        elif triple[1] == ':instance':
            lf += ' & ' + instance_assignment(triple, free)[0]
            i += 1
            
        # content assignment
        elif triple[1] == ':content':
            cont_lf, free, i = cont_assignment(graph, free, tree_depth, i, store)
            lf += ' & ' + cont_lf
            
        # role assignment
        else:
            lf += ' & ' + role_assignment(triple)
            i += 1

    # Remove surplus leading conjunction
    lf = lf[3:]

    return lf, free, store


def cont_assignment(graph, free, tree_depth, i, store):
    '''
    Takes graph, free, tree_depth info, index of :content triple, and store.
    Determines scope of :content.
    Translates :content scope subgraph.
    Returns LF, free, and index for end of subgraph.
    '''
    
    triple = graph[i]
    i += 1

    # Build :content subgraph by iterating through remaining triples, if depth < or = that of triple, break
    subgraph = []

    for remaining_triple in graph[i:]:
        if tree_depth[remaining_triple] <= tree_depth[triple]:
            break
        else: 
            subgraph.append(remaining_triple)
            i += 1

    # Translate subgraph
    lf, content_free, store = translation_func(subgraph, set(), tree_depth, store)

    content_lf = close(lf, content_free, store)[0]
    lf = f'cont({triple[0]})({content_lf})'

    free -= content_free

    return lf, free, i


## Examples

Example `ex3` and `ex4` translate to the same LF. We can either add a close operation on `ex3`, or assume a dummy scope node at the root of every AMR as in `ex4`.

In [10]:
ex1 = '(s / SCOPE :ARG0 b :pred (d / dance-01 :ARG0 (b / boy :quant every)))'
ex2 = '(s / SCOPE :ARG0 v :pred (h / hope-01 :ARG0 (b / boy) :content (b2 / buy-01 :ARG0 h :ARG1 (v / violin :quant a))))'
ex3 = '(t / think-01 :ARG0 (b / boy) :content (s / SCOPE :ARG0 v :pred (h / hope-01 :ARG0 (g / girl) :content (b2 / buy-01 :ARG0 g :ARG1 (v / violin :quant a)))))'
ex4 = '(s / SCOPE :pred (t / think-01 :ARG0 (b / boy) :content (s2 / SCOPE :ARG0 v :pred (h / hope-01 :ARG0 (g / girl) :content (b2 / buy-01 :ARG0 g :ARG1 (v / violin :quant a))))))'

In [11]:
for ex in [ex1,ex2,ex3,ex4]:
    g = penman.decode(ex)
    tree_depth = node_depth(g.epidata)
    graph = g.triples
    free = set()

    # Quant store
    graph, store = quant_store(graph, tree_depth)

    # Translate
    lf, free, store = translation_func(graph, free, tree_depth, store)

    # Close if root node is not scope node
    if free:
        lf = close(lf, free, store)[0]

    print(f'⟦{ex}⟧')
    print('=' + lf)
    print('-' * 20)

⟦(s / SCOPE :ARG0 b :pred (d / dance-01 :ARG0 (b / boy :quant every)))⟧
=every(λb.boy(b))(λb.∃d.dance-01(d) & ARG0(d)(b))
--------------------
⟦(s / SCOPE :ARG0 v :pred (h / hope-01 :ARG0 (b / boy) :content (b2 / buy-01 :ARG0 h :ARG1 (v / violin :quant a))))⟧
=a(λv.violin(v))(λv.∃b,h.hope-01(h) & ARG0(h)(b) & boy(b) & cont(h)(∃b2.buy-01(b2) & ARG0(b2)(h) & ARG1(b2)(v)))
--------------------
⟦(t / think-01 :ARG0 (b / boy) :content (s / SCOPE :ARG0 v :pred (h / hope-01 :ARG0 (g / girl) :content (b2 / buy-01 :ARG0 g :ARG1 (v / violin :quant a)))))⟧
=∃b,t.think-01(t) & ARG0(t)(b) & boy(b) & cont(t)(a(λv.violin(v))(λv.∃g,h.hope-01(h) & ARG0(h)(g) & girl(g) & cont(h)(∃b2.buy-01(b2) & ARG0(b2)(g) & ARG1(b2)(v))))
--------------------
⟦(s / SCOPE :pred (t / think-01 :ARG0 (b / boy) :content (s2 / SCOPE :ARG0 v :pred (h / hope-01 :ARG0 (g / girl) :content (b2 / buy-01 :ARG0 g :ARG1 (v / violin :quant a))))))⟧
=∃b,t.think-01(t) & ARG0(t)(b) & boy(b) & cont(t)(a(λv.violin(v))(λv.∃g,h.hope-01(h) &